In [8]:
"""
    OnChainFX pulls data from Messari so will be refering to their API
    Without an API key, requests are rate limited to 20 requests per minute and 1000 requests per day. 
    Users that create an account will have slightly higher limits of 30 requests per minute and 2000 requests per day. 
    PRO users have the highest limit at 60 requests per minute up to a maximum of 4000 requests per day.
"""



'\n    OnChainFX pulls data from Messari so will be refering to their API\n    Without an API key, requests are rate limited to 20 requests per minute and 1000 requests per day. \n    Users that create an account will have slightly higher limits of 30 requests per minute and 2000 requests per day. \n    PRO users have the highest limit at 60 requests per minute up to a maximum of 4000 requests per day.\n'

In [9]:
pip install -U git+https://github.com/messari/messari-python-api.git

  Cloning https://github.com/messari/messari-python-api.git to /private/var/folders/3x/xv4sp89x49z5ctls1tw24k5c0000gn/T/pip-req-build-08vlju83
  Running command git clone -q https://github.com/messari/messari-python-api.git /private/var/folders/3x/xv4sp89x49z5ctls1tw24k5c0000gn/T/pip-req-build-08vlju83
  Resolved https://github.com/messari/messari-python-api.git to commit 56e7e5e923756ec3eff4028bf9c09e087e6f8475
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
# https://github.com/messari/messari-python-api
from messari.messari import Messari

messari = Messari()

In [11]:
from dataclasses import dataclass

@dataclass
class StableCoin:
    """Class for condensing exchange data to just the props we care about"""
    id: str = ''
    symbol: str = ''
    name: str = ''
    market_cap: float = 0
    circulating_supply: float = 0
    total_supply: float = 0
    twentyfourhour_volume: float = 0
    price: float = 0
    last_updated: str = ''

In [12]:
assets = ['tether', 'usd-coin', 'binance-usd', 'dai', 'usdp', 'gemini-dollar', 'tusd']

In [13]:
asset_metric_df = messari.get_asset_metrics(asset_slugs=assets)

In [15]:
coins = []
for asset in assets:
    coin = StableCoin()
    coin.id = asset
    coins.append(coin)

In [16]:
# https://messari.io/report/messari-proprietary-methods

for coin in coins:
    coin.symbol = asset_metric_df['symbol'][coin.id]
    coin.name = asset_metric_df['name'][coin.id]
    coin.price = asset_metric_df['market_data_price_usd'][coin.id]
    coin.market_cap = asset_metric_df['marketcap_current_marketcap_usd'][coin.id]
    coin.circulating_supply = asset_metric_df['supply_circulating'][coin.id]
    # No other supply metrics return values
    coin.total_supply = 0
    # Note there is also prop 'market_data_real_volume_last_24_hours'
    coin.twentyfourhour_volume = asset_metric_df['market_data_volume_last_24_hours'][coin.id]
    coin.price = asset_metric_df['market_data_price_usd'][coin.id]

In [17]:
import pandas as pd

df = pd.DataFrame(coins)

df

,id,symbol,name,market_cap,circulating_supply,total_supply,twentyfourhour_volume,price,last_updated
0,tether,USDT,Tether,6.543761e+10,6.550665e+10,0,7.669742e+08,0.999014,
1,usd-coin,USDC,USD Coin,4.407677e+10,4.407677e+10,0,1.914089e+08,1.000000,
2,binance-usd,BUSD,Binance USD,2.275918e+10,2.275918e+10,0,5.404959e+08,1.000000,
3,dai,DAI,Dai,5.367262e+09,5.368266e+09,0,2.644100e+07,0.999945,
4,usdp,USDP,USDP Stablecoin,2.894624e+06,2.901572e+06,0,2.290113e+02,0.997605,
5,gemini-dollar,GUSD,Gemini Dollar,6.111397e+08,6.111397e+08,0,1.142830e+05,1.000000,
6,tusd,TUSD,TrueUSD,8.471037e+08,8.495032e+08,0,3.221127e+06,0.997400,


In [6]:
# Get all supported coins for OnChainFX

import pandas as pd
dfs = [] # list to hold metric DataFrames
for i in range(1, 5, 1):
    df = messari.get_all_assets(page=1, limit=500, asset_metric='marketcap', to_dataframe=True)
    dfs.append(df)
merged_df = pd.concat(dfs)
print(f'Number of assets in DataFrame {len(merged_df)}')
merged_df.head()

Number of assets in DataFrame 2000


,slug,metrics_marketcap_rank,metrics_marketcap_marketcap_dominance_percent,metrics_marketcap_current_marketcap_usd,metrics_marketcap_y_2050_marketcap_usd,metrics_marketcap_y_plus10_marketcap_usd,metrics_marketcap_liquid_marketcap_usd,metrics_marketcap_volume_turnover_last_24_hours_percent,metrics_marketcap_realized_marketcap_usd,metrics_marketcap_outstanding_marketcap_usd
bitcoin,bitcoin,1,37.392975,3.180852e+11,3.474167e+11,3.427802e+11,3.181844e+11,0.959776,3.895947e+11,3.184747e+11
ethereum,ethereum,2,17.261932,1.468395e+11,1.621526e+11,1.478226e+11,1.400461e+11,1.066192,1.518320e+11,1.415430e+11
tether,tether,3,7.692819,6.543934e+10,NaN,NaN,NaN,NaN,8.876829e+08,8.870120e+08
binance-coin,binance-coin,4,5.539402,4.712119e+10,2.945581e+10,3.191409e+10,3.191409e+10,0.736864,NaN,NaN
usd-coin,usd-coin,5,5.181510,4.407677e+10,NaN,NaN,NaN,NaN,3.978798e+10,4.027128e+10


In [ ]:
merged_df['slug'].to_excel('OnChainFX_Coins.xlsx')